# 推理机制 (Inference)

**目标：** 从数学和直观的角度理解大语言模型（LLM）是如何工作的。我们将忽略底层的工程代码实现，重点关注数据如何在模型中流动，以及“文本”是如何转化为“矩阵运算”的。

**前置知识：** 线性代数。

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from model.model_minimind import MiniMindConfig
from trainer.trainer_utils import get_lr, Logger, is_main_process, lm_checkpoint, init_distributed_mode, setup_seed, init_model, SkipBatchSampler

# 设置绘图风格
sns.set_theme(style="whitegrid")
plt.rcParams['font.sans-serif'] = ['SimHei'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完毕！")


lm_config = MiniMindConfig(hidden_size=512, num_hidden_layers=8, use_moe=False)
model, tokenizer = init_model(lm_config, "pre_train", "./model", "./out")

## 1. 从文本到数字：Tokenization (分词)

计算机无法直接理解“苹果”或“Apple”这样的字符，它只能处理数字。第一步是将人类语言离散化。

### 1.1 什么是 Token？
Token 是模型处理的最小单位，它可以是一个词语，也可以是单字。

In [ ]:
# 加载分词器
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 输入一个句子
text = "The mathematics of deep learning is beautiful."

# 分词过程
tokens = tokenizer.tokenize(text)
ids = tokenizer.encode(text)

print(f"原始文本: {text}")
print(f"分词结果 (Tokens): {tokens}")
print(f"数字索引 (IDs):   {ids}")